# *Project 2: Credit Card Dataset*


We try to build a machine learning model to predict if an applicant is 'good' or 'bad' client. <br>
The definition of 'good' or 'bad' is not given, so we differentiate it ourselves. 

The dataset contains two csv files, which can be merged by client ID: <br>
application_record.csv <br>
credit_record.csv <br>





Project Overview

## Importing Relevant Libraries

In [448]:
%matplotlib inline
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

# General math, csv and plotting modules.
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt

# Widget and formatting modules
import ipywidgets
from ipywidgets import interact, interactive, interact_manual
import pylab
import seaborn as sns
pylab.rcParams['figure.figsize'] = (10, 5)

# Visualization Libraries
import io
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
from mpl_toolkits.mplot3d import Axes3D
from plotly.subplots import make_subplots
from matplotlib.pyplot import figure

# Loading the required libraries 
from fancyimpute import IterativeImputer as MICE
import sklearn
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import cohen_kappa_score
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from yellowbrick.classifier import DiscriminationThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, ExpSineSquared
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import Image

# Avoiding warnings
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from scipy.stats import norm, skew #for some statistics
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read Dataset

In [449]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [450]:
data = pd.read_csv('/content/drive/MyDrive/ML4EconProject/application_record.csv') 
data.head()
data.shape

(438557, 18)

We observe some duplicate values and we drop them.

In [451]:
data.loc[data.DAYS_EMPLOYED==-1194].loc[data.DAYS_BIRTH== -10554]

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
213,5009031,F,N,Y,0,315000.000,Working,Secondary / secondary special,Single / not married,House / apartment,-10554,-1194,1,0,0,0,Accountants,1.000
214,5009032,F,N,Y,0,315000.000,Working,Secondary / secondary special,Single / not married,House / apartment,-10554,-1194,1,0,0,0,Accountants,1.000
215,6153669,F,N,Y,0,315000.000,Working,Secondary / secondary special,Single / not married,House / apartment,-10554,-1194,1,0,0,0,Accountants,1.000


In [452]:
data=data.drop_duplicates(subset=data.columns[1:], keep='first', inplace=False)
data

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.000,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.000
2,5008806,M,Y,Y,0,112500.000,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.000
3,5008808,F,N,Y,0,270000.000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.000
7,5008812,F,N,Y,0,283500.000,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.000
10,5008815,M,Y,Y,0,270000.000,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438541,6837707,M,N,Y,0,202500.000,Working,Higher education,Civil marriage,House / apartment,-13510,-2309,1,1,0,0,Laborers,2.000
438545,6839651,F,N,Y,3,99000.000,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-18832,365243,1,0,0,0,NaN,1.000
438547,6839917,F,N,Y,0,180000.000,Pensioner,Higher education,Married,House / apartment,-10966,-2704,1,0,0,0,NaN,2.000
438552,6840104,M,N,Y,0,135000.000,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.000


In [453]:
credit=pd.read_csv('/content/drive/MyDrive/ML4EconProject/credit_record.csv') 
credit

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


Information about STATUS: <br>
 0: 1-29 days past due <br>
 1: 30-59 days past due <br>
 2: 60-89 days overdue <br>
  3: 90-119 days overdue <br>
  4: 120-149 days overdue  <br>
  5: Overdue or bad debts, write-offs for more than 150 days  <br>
  C: paid off that month <br>
  X: No loan for the month <br>

We consider users who overdue for more than 60 days as target risk users. We label them as '1'. 

In [454]:
credit['label'] = None
credit['label'][credit['STATUS'] =='2']= 'Yes'
credit['label'][credit['STATUS'] =='3']='Yes' 
credit['label'][credit['STATUS'] =='4']='Yes' 
credit['label'][credit['STATUS'] =='5']='Yes' 

In [455]:
df=credit.groupby('ID').count()
df['label'][df['label'] > 0]=1 
df['label'][df['label'] == 0]=0 
df = df[['label']]

In [456]:
print(df['label'].value_counts())

0    45318
1      667
Name: label, dtype: int64


In [457]:
df=pd.merge(data,df,how='inner',on='ID')
df

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,label
0,5008804,M,Y,Y,0,427500.000,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.000,0
1,5008806,M,Y,Y,0,112500.000,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.000,0
2,5008808,F,N,Y,0,270000.000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.000,0
3,5008812,F,N,Y,0,283500.000,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.000,0
4,5008815,M,Y,Y,0,270000.000,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,5148694,F,N,N,0,180000.000,Pensioner,Secondary / secondary special,Civil marriage,Municipal apartment,-20600,-198,1,0,0,0,Laborers,2.000,1
9705,5149055,F,N,Y,0,112500.000,Commercial associate,Secondary / secondary special,Married,House / apartment,-15837,-2694,1,1,1,0,NaN,2.000,1
9706,5149729,M,Y,Y,0,90000.000,Working,Secondary / secondary special,Married,House / apartment,-19101,-1721,1,0,0,0,NaN,2.000,1
9707,5149838,F,N,Y,0,157500.000,Pensioner,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.000,1


In [458]:
data.shape

(90085, 18)

In [459]:
df.columns

Index(['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'label'],
      dtype='object')

We change the names of the columns for better understanding.

In [460]:
df.rename(columns={'ID':'ClientID','CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car','FLAG_OWN_REALTY':'Reality',
                   'CNT_CHILDREN':'Children','AMT_INCOME_TOTAL':'Total_Income', 'NAME_INCOME_TYPE':'Income_Type',
                    'NAME_EDUCATION_TYPE':'Education','NAME_FAMILY_STATUS':'Family_Status',
                    'NAME_HOUSING_TYPE':'Housing_Type', 'DAYS_BIRTH':'Birthday', 'DAYS_EMPLOYED':'Employment_Day', 
                   	'FLAG_MOBIL': 'Has_Mobile', 'FLAG_EMAIL':'Email','FLAG_WORK_PHONE':'Workphone',
                    'FLAG_PHONE':'Phone','CNT_FAM_MEMBERS':'Family_Size',
                   'OCCUPATION_TYPE':'Occupation' },inplace=True)

In [432]:
df.head()

,ClientID,Gender,Car,Reality,Children,Total_Income,Income_Type,Education,Family_Status,Housing_Type,Birthday,Employment_Day,Has_Mobile,Workphone,Phone,Email,Occupation,Family_Size,label
0,5008804,M,Y,Y,0,427500.000,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.000,0
1,5008806,M,Y,Y,0,112500.000,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.000,0
2,5008808,F,N,Y,0,270000.000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.000,0
3,5008812,F,N,Y,0,283500.000,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.000,0
4,5008815,M,Y,Y,0,270000.000,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.000,0


In [433]:
df.describe(include='all') 

,ClientID,Gender,Car,Reality,Children,Total_Income,Income_Type,Education,Family_Status,Housing_Type,Birthday,Employment_Day,Has_Mobile,Workphone,Phone,Email,Occupation,Family_Size,label
count,9709.000,9709,9709,9709,9709.000,9709.000,9709,9709,9709,9709,9709.000,9709.000,9709.000,9709.000,9709.000,9709.000,6715,9709.000,9709.000
unique,nan,2,2,2,nan,nan,5,5,5,6,nan,nan,nan,nan,nan,nan,18,nan,nan
top,nan,F,N,Y,nan,nan,Working,Secondary / secondary special,Married,House / apartment,nan,nan,nan,nan,nan,nan,Laborers,nan,nan
freq,nan,6323,6139,6520,nan,nan,4960,6761,6530,8684,nan,nan,nan,nan,nan,nan,1724,nan,nan
mean,5076104.679,NaN,NaN,NaN,0.423,181228.195,NaN,NaN,NaN,NaN,-15991.812,61732.846,1.000,0.217,0.288,0.088,NaN,2.183,0.021
std,40802.696,NaN,NaN,NaN,0.767,99277.305,NaN,NaN,NaN,NaN,4246.224,139656.323,0.000,0.413,0.453,0.283,NaN,0.933,0.144
min,5008804.000,NaN,NaN,NaN,0.000,27000.000,NaN,NaN,NaN,NaN,-25152.000,-15713.000,1.000,0.000,0.000,0.000,NaN,1.000,0.000
25%,5036955.000,NaN,NaN,NaN,0.000,112500.000,NaN,NaN,NaN,NaN,-19565.000,-2995.000,1.000,0.000,0.000,0.000,NaN,2.000,0.000
50%,5069449.000,NaN,NaN,NaN,0.000,157500.000,NaN,NaN,NaN,NaN,-15611.000,-1374.000,1.000,0.000,0.000,0.000,NaN,2.000,0.000
75%,5112986.000,NaN,NaN,NaN,1.000,225000.000,NaN,NaN,NaN,NaN,-12440.000,-339.000,1.000,0.000,1.000,0.000,NaN,3.000,0.000


In [434]:
df.dropna(axis=0, inplace=True)

In [435]:
df.describe(include=[np.number])

,ClientID,Children,Total_Income,Birthday,Employment_Day,Has_Mobile,Workphone,Phone,Email,Family_Size,label
count,6715.000,6715.000,6715.000,6715.000,6715.000,6715.000,6715.000,6715.000,6715.000,6715.000,6715.000
mean,5076510.377,0.508,189606.358,-14769.037,-2485.386,1.000,0.267,0.287,0.097,2.282,0.020
std,40919.491,0.819,102224.724,3529.228,2299.573,0.000,0.442,0.452,0.296,0.977,0.139
min,5008806.000,0.000,27000.000,-24611.000,-15713.000,1.000,0.000,0.000,0.000,1.000,0.000
25%,5036962.000,0.000,126000.000,-17448.000,-3350.500,1.000,0.000,0.000,0.000,2.000,0.000
50%,5078898.000,0.000,166500.000,-14548.000,-1788.000,1.000,0.000,0.000,0.000,2.000,0.000
75%,5113032.500,1.000,225000.000,-11919.500,-859.000,1.000,1.000,1.000,0.000,3.000,0.000
max,5150467.000,19.000,1575000.000,-7489.000,-17.000,1.000,1.000,1.000,1.000,20.000,1.000


## Data Cleaning

There are 7 binary variables in the dataset. Namely: Gender, Car, Realty, Mobile, Workphone, Phone and Email. However, since every applicant has a mobile, we drop column Mobile. <br>


In [436]:
df= df.drop(['Has_Mobile'], axis=1)

We divide each employment day by 365 in order to have the feauture as years.(keeping only the float number)

In [437]:
df['Employment_Day']=-df['Employment_Day']//365


In [438]:
df=df.rename(columns={"Employment_Day": "Employment_Years"})

We convert Birth days to age number.

In [439]:
df['Birthday']= - df['Birthday'] //365

In [440]:
df=df.rename(columns={"Birthday": "Age"})

In [441]:
df.head(5)

,ClientID,Gender,Car,Reality,Children,Total_Income,Income_Type,Education,Family_Status,Housing_Type,Age,Employment_Years,Workphone,Phone,Email,Occupation,Family_Size,label
1,5008806,M,Y,Y,0,112500.000,Working,Secondary / secondary special,Married,House / apartment,58,3,0,0,0,Security staff,2.000,0
2,5008808,F,N,Y,0,270000.000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52,8,0,1,1,Sales staff,1.000,0
4,5008815,M,Y,Y,0,270000.000,Working,Higher education,Married,House / apartment,46,2,1,1,1,Accountants,2.000,0
5,5008819,M,Y,Y,0,135000.000,Commercial associate,Secondary / secondary special,Married,House / apartment,48,3,0,0,0,Laborers,2.000,0
6,5008825,F,Y,N,0,130500.000,Working,Incomplete higher,Married,House / apartment,29,3,0,0,0,Accountants,2.000,0


## Initial EDA

In [442]:
df.groupby("Gender")["Gender"].count()

Gender
F    4039
M    2676
Name: Gender, dtype: int64

We observe that 4039/2676= 60.15% of the clients are female, as it can also be seen in the below plot.

In [443]:
import plotly.express as px
fig = px.pie(df, names="Gender", title='Gender',width=400, height=400)

fig.show(renderer="colab")

From them, 6583 are classified as "good" clients, and 132 as "bad" clients. <br>
However, this creates class imbalance problem in the dataset, since only 2% of the clients are classified as risk clients. 

In [444]:
df.groupby("label")["label"].count()

label
0    6583
1     132
Name: label, dtype: int64

88% of clients have house or apartment as housing_type and both co-op apartment and office apartment have the lowest percentage, 0.4% and 0.9% respectively.

In [445]:
df.groupby("Housing_Type")["Housing_Type"].count()

Housing_Type
Co-op apartment          28
House / apartment      5922
Municipal apartment     233
Office apartment         58
Rented apartment        112
With parents            362
Name: Housing_Type, dtype: int64

In [446]:
fig = px.pie(df, names="Housing_Type", title='Housing_Type',width=800, height=400)

fig.show(renderer="colab")

In [447]:
housing_df = df.Housing_Type.value_counts(normalize = True)
housing_df

House / apartment     0.882
With parents          0.054
Municipal apartment   0.035
Rented apartment      0.017
Office apartment      0.009
Co-op apartment       0.004
Name: Housing_Type, dtype: float64

We can see that from "risk" clients, none of men have rented apartment, municipal apartment and co-op apartment. Also none of women clients have office apartment.

In [464]:
fig = px.bar(df, x="Gender",y="label", color="Housing_Type", barmode="group",
labels={
                     "label": "occurrences"                     
                 }

)
fig.show(renderer="colab")

In [ ]:
fig = px.bar(df, x="Gender",y="label", color="Education", barmode="group")
fig.show(renderer="colab")

In [ ]:
fig = px.bar(df, x="Education", y="label", color="Occupation", barmode="group")
fig.show(renderer="colab")

In [ ]:
family = df.Family_Status.value_counts()
sns.set(style="darkgrid")
sns.barplot(family.index, family.values, alpha=0.9)
plt.title('Frequency Distribution of Family Status')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Status', fontsize=12)
plt.show()

In [ ]:
income = df.Income_Type.value_counts()
sns.barplot(income.index, income.values, alpha=0.9)
plt.title('Frequency Distribution of Income Status')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Income', fontsize=12)
plt.show()

In [ ]:
occup = df.Occupation.value_counts()
sns.barplot( occup.values, occup.index,alpha=0.9)
plt.title('Frequency Distribution of Occupation')
plt.ylabel('Occupation', fontsize=12)
plt.xlabel('Number of Occurrences', fontsize=12)
plt.show( )

In [ ]:
fam_size = df.Family_Size.value_counts()
sns.barplot(fam_size.index, fam_size.values, alpha=0.9)
plt.title('Frequency Distribution of Family Size')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Family Size', fontsize=12)
plt.show()

## Data Preprocessing

We replace Male and Female denoted as "M" and "F" with 1 and 0 accordingly.
We replace Yes and No denoted as "Y" and "N" with 1 and 0 accordingly. And we make Workphone,Phone and Email as factor.

In [ ]:
df= df.replace('M', 1)
df= df.replace('F', 0)
df= df.replace('Y', 1)
df= df.replace('N', 0)

df['Email'] = pd.factorize(df.Email)[0]
df['Phone'] = pd.factorize(df.Phone)[0]
df['Workphone'] = pd.factorize(df.Workphone)[0]

In [ ]:
df.head(5)

In [ ]:
X = df.iloc[:,1:-1]
X = pd.get_dummies(X)
X

# Getting the target variable
y = df.iloc[:,-1]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In this dataset we are dealing with 5 categorical variables, namely Income_Type,	Education,	Family_Status,	Housing_Type and Occupation. We need to transform them into numerical values for further EDA and model building. We generate a label encoding scheme for mapping each category to a numeric value.<br>
As encoding option, we use one-hot encoding. This will increase the feature space from 17 to 51 . This inherently induces a Dimensionality problem that will cause overfitting ( We will resolved it through PCA)

In [ ]:
sc = StandardScaler()
X = sc.fit_transform(X)

## Balancing using SMOTE

As we said, there is class imbalance problem in the dataset, since only 2% of the clients are classified as risk clients.

In [ ]:
x_df = pd.DataFrame(X)
y_df = pd.DataFrame(y, columns=['Target'])
df_full = pd.concat([x_df,y_df],axis =1, sort = False)

In [ ]:
## Redefining the 1 and 0 dataset after preprocessing 
T_0 = df_full[df_full['Target']==0]
T_1 = df_full[df_full['Target']==1]
print(T_0.shape, T_1.shape)

# # Plotting the imbLanced dataset
figure(num=None, figsize=(8, 5), dpi=80, facecolor='w', edgecolor='k')
sns.distplot(df_full['Target'], hist=False, kde=True, 
             bins=2, color = 'blue',
             hist_kws={"rwidth":1.0,'edgecolor':'black'})
# Add labels
plt.title('Original Imbalanced Distribution in Dataset')
plt.xlabel('Good vs Bad')
plt.ylabel('Density')

In [ ]:
# Setting the target and feature space for balanced dataset
X = df_full.loc[:, df_full.columns != 'Target']
Y = df_full['Target']

# Handling Oversampling for the given dataset using SMOTE
smt = SMOTE(random_state=42)
x_smt, y_smt = smt.fit_sample(X,Y)

In [ ]:
# Defining the target and train variable
Y_train = pd.DataFrame({'Target': y_smt[0:, ]})['Target']
X_train = pd.DataFrame(data = x_smt,
                 index=[i for i in range(x_smt.shape[0])],
                 columns=['f'+ str(i) for i in range(x_smt.shape[1])])

In [ ]:
# After Oversampling we plot the baLanced dataset
figure(num=None, figsize=(8, 5), dpi=80, facecolor='w', edgecolor='k')
sns.distplot(Y_train, hist=False, kde=True, 
             bins=2, color = 'blue',
             hist_kws={"rwidth":1.0,'edgecolor':'black'})
# Add labels
plt.title('Distribution of Dataset after SMOTE')
plt.xlabel('Good vs Bad')
plt.ylabel('Density')

## Train and Test split

In [ ]:
#i split to 40% in order to reduce overfitting
X_train, X_test, Y_train, y_test = train_test_split(X_train, Y_train, test_size=0.25, random_state=0)

## PCA (maybe)

In order to avoid redundant information of features, we generate a new set of variables (principal components). Each principal component is a linear combination of the original variables. We know that principal components explain a part of the variance. After setting n_components=2, we observe that the PCA plot shows 2 separate clusters of samples and almost clearly distinguish between good and bad client. Also, the explained variance of only 2 components is 100% so there is no need to add another component.

In [ ]:
"""pca = PCA(n_components=2)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

pca.fit(X_train)
int_y_train = pd.DataFrame(y_train, columns= ["Good/Bad Client"])
principalDf = pd.DataFrame(data = X_train
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, int_y_train], axis = 1)

fig = plt.figure(figsize = (10,7))
ax = plt.axes(projection='3d')
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_zlabel("Good/Bad Client")
ax.set_title('2 component PCA', fontsize = 20)
targets = [0, 1]
colors = ['r', 'g']
ax.scatter3D(finalDf['principal component 1'], finalDf['principal component 2'], finalDf["Good/Bad Client"],
             c = finalDf["Good/Bad Client"], cmap="Set2_r",s = 60)
ax.grid()

print('Explained variance: {}'.format(np.sum(pca.explained_variance_ratio_)))"""

Typically, we want the explained variance to be between 95–99%. So we experimented by setting the n_components=0.95 and n_components=0.99. Unfortunately, we observed that all the classifiers still overfit and we could not overpass the dimensionality problem. 

## Defining Functions for Plotting

In [ ]:
# Building some visualization functions to be used for each model as well as model comparisions
def roc_conf_plot(classifier, x_train = None, y_train = None, x_test = None,  y_test = None):
    
    # Fitting the model
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    roc_score = roc_auc_score(y_test, y_pred)
    prob = classifier.predict_proba(x_test)
    
    # Setting traces for the ROC plot
    fpr,tpr,thresholds = roc_curve(y_test,prob[:,1])
    trace1 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(roc_score),
                        line = dict(color = ('purple'),width = 2),
                       )
    trace2 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('red'),width = 2,
                        dash = 'dot'))
    Layout = go.Layout(dict(title = "Receiver Operator Characteristics Curve",
                           showlegend = False,
                           xaxis = dict(title = "False positive rate",
                                         domain=[0, 0.6],
                                         ticklen=5,gridwidth=2),
                            yaxis = dict(title = "True positive rate",
                                         zerolinewidth=1,
                                         ticklen=5,gridwidth=2)
                           )
                           )
    data = [trace1, trace2]
    figure(num=None, figsize=(8, 5), dpi=80, facecolor='w', edgecolor='k')
    fig = go.Figure(data= data, layout = Layout)
    py.iplot(fig)
    print("Area under curve : ", roc_score)

# Generates the threshold plot for the specified model
def threshold_plot(algorithm, x, y):
  figure(num=None, figsize=(8, 5), dpi=80, facecolor='w', edgecolor='k')
  visualizer = DiscriminationThreshold(algorithm)
  visualizer.fit(x,y)
  visualizer.poof()

# Creating row table for the metric comparision of each model.
def model_metric(model, x_train, y_train, x_test, y_test, name):
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    roc = roc_auc_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)


    row_df = pd.DataFrame({
                       "Model"           : [name],
                       "Accuracy_score"  : [acc],
                       "Recall_score"    : [recall],
                       "Precision"       : [prec],
                       "f1_score"        : [f1],
                       "Area_under_curve": [roc],
                     
    })
    return row_df

In [ ]:
def printTestTrainLoss(history, nrOfEpochs):
  loss_train = history.history['loss']
  loss_val = history.history['val_loss']
  epochs = range(1,11)
  plt.plot(epochs, loss_train, 'g', label='Training loss')
  plt.plot(epochs, loss_val, 'b', label='validation loss')
  plt.title('Training and Validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  return plt

In [ ]:
def printTestTrainAccuracy(history, nrOfEpochs):
  loss_train = history.history['accuracy']
  loss_val = history.history['val_accuracy']
  epochs = range(1,11)
  plt.plot(epochs, loss_train, 'g', label='Training accuracy')
  plt.plot(epochs, loss_val, 'b', label='validation accuracy')
  plt.title('Training and Validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  return plt

In [ ]:
def printAccAndLossSideBySide(history, nrOfEpochs):
  fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
  epochs = range(1,11)


  acc_train = history.history['accuracy']
  acc_val = history.history['val_accuracy']
  loss_train = history.history['loss']
  loss_val = history.history['val_loss']

  axes[0].plot(epochs, acc_train, 'g', label='Training accuracy')
  axes[0].plot(epochs, acc_val, 'b', label='validation accuracy')
  axes[0].set_title('Accuracy')
  axes[0].set_xlabel('Epochs')
  axes[0].set_ylabel('Accuracy')
  axes[0].legend()

  axes[1].plot(epochs, loss_train, 'g', label='Training loss')
  axes[1].plot(epochs, loss_val, 'b', label='validation loss')
  axes[1].set_title('Loss')
  axes[1].set_xlabel('Epochs')
  axes[1].set_ylabel('Loss')
  axes[1].legend()
  fig.tight_layout()
  return fig

## Models

### Logistic Regression

In [ ]:
"""# Finding best paramters for Logistic Regression CV
parameters = {'solver':('lbfgs', 'liblinear', 'sag', 'saga'),'fit_intercept':(True,False)}
logreg = LogisticRegressionCV(max_iter = 1000000)
logreg_cv = GridSearchCV(logreg, parameters)
logreg_cv.fit(X_train, Y_train)
print("Tuned hyperparameters :(best parameters) ",logreg_cv.best_params_)"""

In [ ]:
# Fitting the model with the best parameters that were identified above.
model_LCV = LogisticRegression(solver='lbfgs', fit_intercept=True, penalty = 'l2',C= 100.0, max_iter=1000000)
history_LCV = model_LCV.fit(X_train, Y_train)
y_pred_LCV = model_LCV.predict(X_test)

In [ ]:
print(recall_score(y_test, y_pred_LCV))
print(precision_score(y_test, y_pred_LCV))
print(accuracy_score(y_test, y_pred_LCV))
print(confusion_matrix(y_test, y_pred_LCV))
# Printing the classification report of the model.
print(classification_report(y_test, y_pred_LCV))

# Constructing the threshold plot and the ROC plot.
#roc_conf_plot(model_LCV, X_train, Y_train,  X_test, y_test)
#threshold_plot(model_LCV, X_train, Y_train)
LCV_model = model_metric(model_LCV, X_train, Y_train, X_test, y_test, "Logistic Regression CV")

In [ ]:
history_LCV#printTestTrainLoss(history_LCV, )

### Support Vector Machines

In [ ]:
"""# Finding the best parameters for SVM
parameters = {'kernel':( 'linear','rbf','sigmoid'), 'C':[1e-2,1e-1,1e0,1e1,1e2,3,10]}
model_SVC = SVC()
svclassifier_cv = GridSearchCV(model_SVC,parameters)
svclassifier_cv.fit(X_train, Y_train)
print("Tuned hyperparameters :(best parameters) ",svclassifier_cv.best_params_)"""

In [ ]:
# Use the best parameters to fit the model
model_SVC = SVC(kernel='rbf',C=100, probability = True)
history_SVC=model_SVC.fit(X_train, Y_train)
model_SVC.score(X_test, y_test)
y_pred_SVC = model_SVC.predict(X_test)

In [ ]:
print(recall_score(y_test, y_pred_SVC))
print(precision_score(y_test, y_pred_SVC))
print(accuracy_score(y_test, y_pred_SVC))
print(confusion_matrix(y_test, y_pred_SVC))

# Printing the classification report of the model.
print(classification_report(y_test,y_pred_SVC))

# Constructing the threshold plot and the ROC plot.
#roc_conf_plot(model_SVC, X_train, Y_train, X_test, y_test)
# threshold_plot(model_SVC, X_train, Y_train)
SVM_model = model_metric(model_SVC, X_train, Y_train, X_test, y_test, "Support Vector Machine")

In [ ]:
history_SVC

### Decision Trees

In [ ]:
parameters = {'criterion':('gini','entropy'),'max_depth': range(1,20,1)}
dectree = DecisionTreeClassifier()
dectree_cv = GridSearchCV(dectree,parameters)
dectree_cv.fit(X_train,Y_train)
print("Tuned hyperparameters :(best parameters) ",dectree_cv.best_params_)

In [ ]:
# Use the best parameters to fit the model
model_DT = DecisionTreeClassifier(criterion="entropy",random_state=0, max_depth=18)
model_DT.fit(X_train,Y_train)
y_pred_DT = model_DT.predict(X_test)

In [ ]:
export_graphviz(model_DT, out_file='tree_max.dot', feature_names = X_train.columns,
                rounded = True, proportion = False, precision = 2, filled = True)

In [ ]:
!dot -Tpng tree_max.dot -o tree_max.png


In [ ]:
# Display in jupyter notebook
Image(filename = 'tree_max.png')

In [ ]:
# Use the best parameters to fit the model
model_DT = DecisionTreeClassifier(criterion="entropy",random_state=0, max_depth=3)
model_DT.fit(X_train,Y_train)
y_pred_DT = model_DT.predict(X_test)
plt.figure(figsize=(25,10))
tree.plot_tree(model_DT);

In [ ]:
export_graphviz(model_DT, out_file='tree_limited.dot', feature_names = X_train.columns,
                rounded = True, proportion = False, precision = 2, filled = True)

In [ ]:
!dot -Tpng tree_limited.dot -o tree_limited.png

In [ ]:
# Display in jupyter notebook
Image(filename = 'tree_limited.png')

In [ ]:
print(recall_score(y_test, y_pred_DT))
print(precision_score(y_test, y_pred_DT))
print(accuracy_score(y_test, y_pred_DT))
print(confusion_matrix(y_test,y_pred_DT))
print(classification_report(y_test,y_pred_DT))

# Constructing the threshold plot and the ROC plot
#roc_conf_plot(model_DT, X_train, Y_train, X_test, y_test)
#threshold_plot(model_DT, X_train, Y_train)
DT_model = model_metric(model_DT, X_train, Y_train, X_test,y_test, "Decision Tree")

### Random Forest

In [ ]:
parameters= {'n_estimators':[10,50,100],'max_depth': range(1,20,2)}
RFC = RandomForestClassifier()
RFC_cv = GridSearchCV(RFC,parameters)
RFC_cv.fit(X_train,Y_train)
print("Tuned hyperparameters :(best parameters) ",RFC_cv.best_params_)

In [ ]:
model_RFC = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=1)
model_RFC.fit(X_train, Y_train)
y_pred_RFC = model_RFC.predict(X_test)

In [ ]:
print(recall_score(y_test, y_pred_RFC))
print(precision_score(y_test, y_pred_RFC))
print(accuracy_score(y_test, y_pred_RFC))
print(confusion_matrix(y_test,y_pred_RFC))
print(classification_report(y_test,y_pred_RFC)) 

# Constructing the threshold plot and the ROC plot
#roc_conf_plot(model_RFC, X_train, Y_train, X_test, y_test)
#threshold_plot(model_RFC, X_train, Y_train)
RF_model = model_metric(model_RFC, X_train, Y_train, X_test, y_test, "Random Forest")

### Neural Network

In [ ]:
# Building the NN architecture
model_NN = Sequential()
classes = 1
model_NN.add(Dense(24,activation='relu',input_dim=50,kernel_initializer='glorot_uniform', bias_initializer='zeros')) 
model_NN.add(Dense(24,activation='relu',kernel_initializer='glorot_uniform', bias_initializer='zeros')) 
model_NN.add(Dense(classes, activation='sigmoid'))

model_NN.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Training the neural network
np.random.seed(7)
history = model_NN.fit(X_train, Y_train, shuffle=True,
          batch_size=5, epochs=10,verbose=1,validation_split=0.2)
y_pred_NN = model_NN.predict(X_test)
y_pred_NN =(y_pred_NN > 0.5)   
eval_model = model_NN.evaluate(X_test, y_test, verbose=0)

In [ ]:
# Printing the classification report of the model.
print(classification_report(y_test,y_pred_NN))

# Getting the confusion matrix of the NN separately.
cmap = sns.cm.rocket_r
sns.set(font_scale=1.4)
conf_df = pd.DataFrame(confusion_matrix(y_test,y_pred_NN).tolist(), columns = ['Positive','Negative'], index = ['Positive','Negative'])
sns.heatmap(conf_df, annot=True, annot_kws={"size": 16}, fmt=".0f", vmin=-0.05, cmap = cmap)
plt.savefig('/content/drive/MyDrive/ML4EconProject/sns.png')
plt.show()

# Plotting the AUC curve
roc_score = roc_auc_score(y_test, y_pred_NN)
prob = model_NN.predict_proba(X_test)

# Setting traces for the ROC plot
fpr,tpr,thresholds = roc_curve(np.array(y_test), prob)
trace1 = go.Scatter(x = fpr,y = tpr,
                    name = "Roc : " + str(roc_score),
                    line = dict(color = ('purple'),width = 2),
                   )
trace2 = go.Scatter(x = [0,1],y=[0,1],
                    line = dict(color = ('red'),width = 2,
                    dash = 'dot'))
Layout = go.Layout(dict(title = "Receiver Operator Characteristics Curve",
                       showlegend = False,
                       xaxis = dict(title = "False positive rate",
                                     domain=[0, 0.6],
                                     ticklen=5,gridwidth=2),
                        yaxis = dict(title = "True positive rate",
                                     zerolinewidth=1,
                                     ticklen=5,gridwidth=2)
                       )
                       )
data = [trace1, trace2]
figure(num=None, figsize=(8, 5), dpi=80, facecolor='w', edgecolor='k')
fig = go.Figure(data= data, layout = Layout)
py.iplot(fig)
print("Area under curve : ", roc_score)

# Collecting model metrics for Neural Network
NN_df = pd.DataFrame({
                       "Model"           : ["Neural Network"],
                       "Accuracy_score"  : [accuracy_score(y_test,y_pred_NN)],
                       "Recall_score"    : [recall_score(y_test,y_pred_NN)],
                       "Precision"       : [precision_score(y_test,y_pred_NN)],
                       "f1_score"        : [f1_score(y_test,y_pred_NN)],
                       "Area_under_curve": [roc_auc_score(y_test, y_pred_NN)],
                      
})

In [ ]:
pltAccuracy = printTestTrainAccuracy(history, 11)
pltAccuracy.savefig('/content/drive/MyDrive/ML4EconProject/NN_accuracy.png')
pltAccuracy.show()
pltLoss = printTestTrainLoss(history, 11)
pltLoss.savefig('/content/drive/MyDrive/ML4EconProject/NN_loss.png')
pltLoss.show()

In [ ]:
fig = printAccAndLossSideBySide(history, 11)
fig.savefig('/content/drive/MyDrive/ML4EconProject/NN_lossAndAcc.png')
fig.show()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model_NN, to_file='/content/drive/MyDrive/ML4EconProject/model_plot.png', show_shapes=True, show_layer_names=True)

###Neural Network with feature selection

In order to reduce the runtime we reduce the list of features using f_classit feature selection.

By removing 10 features we achieve an accuracy similar to the previous model with less time, which can be detrimental for larger datasets

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, chi2

selector = SelectKBest(f_classif, k=50)
selected_features = selector.fit_transform(X_train, Y_train)

X_train2 = X_train.iloc[:, (-selector.scores_).argsort()[:40]]
X_train2.head

X_test2 = X_test.iloc[:, (-selector.scores_).argsort()[:40]]
X_test2.head

In [ ]:
df.columns

In [ ]:
# Building the NN architecture
model_NN2 = Sequential()
classes = 1
model_NN2.add(Dense(24,activation='relu',input_dim=40,kernel_initializer='glorot_uniform', bias_initializer='zeros')) 
model_NN2.add(Dense(24,activation='relu',kernel_initializer='glorot_uniform', bias_initializer='zeros')) 
model_NN2.add(Dense(classes, activation='sigmoid'))

model_NN2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Training the neural network
np.random.seed(7)
history2 = model_NN2.fit(X_train2, Y_train, shuffle=True,
          batch_size=5, epochs=10,verbose=1,validation_split=0.2)
y_pred_NN2 = model_NN2.predict(X_test2)
y_pred_NN2 =(y_pred_NN2 > 0.5)   
eval_model2 = model_NN2.evaluate(X_test2, y_test, verbose=0)

In [ ]:
# Printing the classification report of the model.
print(classification_report(y_test,y_pred_NN2))

# Getting the confusion matrix of the NN separately.
cmap = sns.cm.rocket_r
sns.set(font_scale=1.4)
conf_df = pd.DataFrame(confusion_matrix(y_test,y_pred_NN2).tolist(), columns = ['Positive','Negative'], index = ['Positive','Negative'])
sns.heatmap(conf_df, annot=True, annot_kws={"size": 16}, fmt=".0f", vmin=-0.05, cmap = cmap)
plt.savefig('/content/drive/MyDrive/ML4EconProject/sns_feature_sel.png')
plt.show()

# Plotting the AUC curve
roc_score = roc_auc_score(y_test, y_pred_NN2)
prob = model_NN2.predict_proba(X_test2)

# Setting traces for the ROC plot
fpr,tpr,thresholds = roc_curve(np.array(y_test), prob)
trace1 = go.Scatter(x = fpr,y = tpr,
                    name = "Roc : " + str(roc_score),
                    line = dict(color = ('purple'),width = 2),
                   )
trace2 = go.Scatter(x = [0,1],y=[0,1],
                    line = dict(color = ('red'),width = 2,
                    dash = 'dot'))
Layout = go.Layout(dict(title = "Receiver Operator Characteristics Curve",
                       showlegend = False,
                       xaxis = dict(title = "False positive rate",
                                     domain=[0, 0.6],
                                     ticklen=5,gridwidth=2),
                        yaxis = dict(title = "True positive rate",
                                     zerolinewidth=1,
                                     ticklen=5,gridwidth=2)
                       )
                       )
data = [trace1, trace2]
figure(num=None, figsize=(8, 5), dpi=80, facecolor='w', edgecolor='k')
fig = go.Figure(data= data, layout = Layout)
py.iplot(fig)
print("Area under curve : ", roc_score)

# Collecting model metrics for Neural Network
NN_df2 = pd.DataFrame({
                       "Model"           : ["Neural Network with feature sel."],
                       "Accuracy_score"  : [accuracy_score(y_test,y_pred_NN2)],
                       "Recall_score"    : [recall_score(y_test,y_pred_NN2)],
                       "Precision"       : [precision_score(y_test,y_pred_NN2)],
                       "f1_score"        : [f1_score(y_test,y_pred_NN2)],
                       "Area_under_curve": [roc_auc_score(y_test, y_pred_NN2)],
                      
})

In [ ]:
pltAccuracyFea = printTestTrainAccuracy(history2, 11)
pltAccuracyFea.savefig('/content/drive/MyDrive/ML4EconProject/NN_accuracy_fea.png')
pltAccuracyFea.show()
pltLossFea = printTestTrainLoss(history2, 11)
pltLossFea.savefig('/content/drive/MyDrive/ML4EconProject/NN_loss_fea.png')
pltLossFea.show()

In [ ]:
fig = printAccAndLossSideBySide(history2, 11)
fig.savefig('/content/drive/MyDrive/ML4EconProject/NN_lossAndAcc_fea.png')
fig.show()

# Evaluation Metric

## Overall Comparison

In [ ]:
# We used the model comparision function after each model to record model performance
# Now concatinating the all models to make a combined dataframe
model_comparison_df = pd.concat([LCV_model, DT_model, SVM_model, RF_model ,NN_df, NN_df2], ignore_index=True,axis= 0, sort= False)
model_comparison_df = model_comparison_df.sort_values('Accuracy_score')
display(model_comparison_df)

## AUC and Accuracy Comparison

In [ ]:
# Plotting Accuracy scores and Area under curve comparison plots
fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                    shared_yaxes=False, vertical_spacing=0.001)

fig.append_trace(go.Bar(
    x = model_comparison_df['Accuracy_score'].tolist(),
    y = model_comparison_df['Model'].tolist(),
    marker=dict(
        color='rgba(50, 171, 96, 0.6)',
        line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1)),
#     size = 15,
    name='Accuracy score of each model',
    orientation='h',
), 1, 1)

fig.append_trace(go.Scatter(
    x= model_comparison_df['f1_score'].tolist(), 
    y= model_comparison_df['Model'].tolist(),
    mode = 'lines+markers',
    line_color = 'rgb(128, 0, 128)',
    name = 'F1 Score of each model',
    textfont = dict(size = 200),
), 1, 2)

fig.update_layout(
    title='Accuracy score and F1-score of each Model',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.85],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=True,
        showticklabels=False,
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=2,
        domain=[0, 0.85],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0.05, 0.42],
        dtick=0.25,
    ),
    xaxis2=dict(
        zeroline=False,
        showline=False,
        range=[0.75, 1.0],
        showticklabels=True,
        showgrid=True,
        domain=[0.47, 1],
        side='top',
        dtick=0.05,
    ),
    legend=dict(x=0.029, y=1.038, font_size=8),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

annotations = []

y_s = np.round(model_comparison_df['Accuracy_score'].tolist(), decimals=4)
y_nw = np.round(model_comparison_df['f1_score'].tolist(), decimals=4)

# Adding labels
for ydn, yd, xd in zip(y_nw, y_s, model_comparison_df['Model']):
    # labeling the scatter savings
    annotations.append(dict(xref='x2', yref='y2',
                            y=xd, x=ydn + 0.02,
                            text='{:.3f}'.format(ydn),
                            font=dict(family='Arial', size=12,
                                      color='rgb(128, 0, 128)'),
                            showarrow=False))
    # labeling the bar net worth
    annotations.append(dict(xref='x1', yref='y1',
                            y=xd, x=yd + 0.08,
                            text='{:.3f}'.format(yd),
                            font=dict(family='Arial', size=12,
                                      color='rgb(128, 0, 128)'),
                            showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.08, y=-0.109,
                        text='Telco Churn Dataset"' +
                             '(Accessed on 05 March 2020)',
                        font=dict(family='Arial', size=10, color='rgb(150,150,150)'),
                        showarrow=False))

fig.update_layout(annotations=annotations)
fig.show()

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ML4EconProject/application_record.csv') 
data.head()
data.shape

In [ ]:
evalData = data
evalData.rename(columns={'ID':'ClientID','CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car','FLAG_OWN_REALTY':'Reality',
                   'CNT_CHILDREN':'Children','AMT_INCOME_TOTAL':'Total_Income', 'NAME_INCOME_TYPE':'Income_Type',
                    'NAME_EDUCATION_TYPE':'Education','NAME_FAMILY_STATUS':'Family_Status',
                    'NAME_HOUSING_TYPE':'Housing_Type', 'DAYS_BIRTH':'Birthday', 'DAYS_EMPLOYED':'Employment_Day', 
                   	'FLAG_MOBIL': 'Has_Mobile', 'FLAG_EMAIL':'Email','FLAG_WORK_PHONE':'Workphone',
                    'FLAG_PHONE':'Phone','CNT_FAM_MEMBERS':'Family_Size',
                   'OCCUPATION_TYPE':'Occupation' },inplace=True)

evalData.describe(include='all') 
evalData= evalData.drop(['Has_Mobile'], axis=1)
evalData['Employment_Day']=-evalData['Employment_Day']//365
evalData=evalData.rename(columns={"Employment_Day": "Employment_Years"})
evalData['Birthday']= - evalData['Birthday'] //365
evalData=evalData.rename(columns={"Birthday": "Age"})
evalData= evalData.replace('M', 1)
evalData= evalData.replace('F', 0)
evalData= evalData.replace('Y', 1)
evalData= evalData.replace('N', 0)

evalData['Email'] = pd.factorize(evalData.Email)[0]
evalData['Phone'] = pd.factorize(evalData.Phone)[0]
evalData['Workphone'] = pd.factorize(evalData.Workphone)[0]

#X = df.iloc[:,1:-1]
X = pd.get_dummies(X)
X

sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
X.shape

In [ ]:
predictions_SVC = model_SVC.predict(X)
predictions_NN = model_NN.predict(X)
predictons_DT = model_DT.predict(X)
predictions_RF = model_RFC.predict(X)
predictions_LR = model_LCV.predict(X)


In [ ]:
predictions_SVC.sum()
yes = (predictions_SVC.sum(), predictions_LR.sum(), predictions_NN.sum(), predictions_RF.sum(), predictons_DT.sum())
yes

In [ ]:
N = 5
risk = (predictions_SVC.sum(), predictions_LR.sum(), predictions_NN.sum(), predictions_RF.sum(), predictons_DT.sum())
noRisk = (len(predictions_SVC)-predictions_SVC.sum(), len(predictions_LR)-predictions_LR.sum(), len(predictions_NN)-predictions_NN.sum(), len(predictions_RF)-predictions_RF.sum(), len(predictons_DT)-predictons_DT.sum())


ind = np.arange(N) # the x locations for the groups
width = 0.35
fig = plt.figure(figsize=(5,5))
ax = fig.add_axes([0,0,1,1])
ax.bar(ind, risk, width, color='r')
ax.bar(ind, noRisk, width,bottom=yes, color='b')
ax.set_ylabel('Number of applicants')
ax.set_title('Prediction of viable applicants')
ax.set_xticks(ind, ('G1', 'G2', 'G3', 'G4', 'G5'))
ax.set_xticklabels(['','SVC','LR','NN','RF','DT'])


#ax.set_yticks(np.arange(0, 81, 10))
#plt.legend(loc='upper right',labels=['Risk', 'No Risk'])
plt.legend(bbox_to_anchor=(1.5, 1), loc='upper right', labels=['Risk', 'No Risk'])
plt.savefig('/content/drive/MyDrive/ML4EconProject/resultsApplicants.png')
plt.show()

In [ ]:
N = 5
gain = 213
riskSavings = -1000
risk = [predictions_SVC.sum(), predictions_LR.sum(), predictions_NN.sum(), predictions_RF.sum(), predictons_DT.sum()]
noRisk = (len(predictions_SVC)-predictions_SVC.sum(), len(predictions_LR)-predictions_LR.sum(), len(predictions_NN)-predictions_NN.sum(), len(predictions_RF)-predictions_RF.sum(), len(predictons_DT)-predictons_DT.sum())
accuracyList = [0.988, 0.693, 0.981, 0.987, 0.974]


risk = [element * riskSavings for element in risk]
noRisk = [element * gain for element in noRisk]
total = np.add(risk, noRisk)

total = np.multiply(total, accuracyList)
#total = [element * riskSavings for element in risk] total.dot(accuracyList)
print(total)
ind = np.arange(N) # the x locations for the groups
width = 0.35
fig = plt.figure(figsize=(5,5))
ax = fig.add_axes([0,0,1,1])
ax.bar(ind, total, width, color='g')
ax.set_ylabel('Net benefit')
ax.set_title('Prediction of net benefit')
ax.set_xticks(ind, ('G1', 'G2', 'G3', 'G4', 'G5'))
ax.set_xticklabels(['','SVC','LR','NN','RF','DT'])


#ax.set_yticks(np.arange(0, 81, 10))
#plt.legend(loc='upper right',labels=['Risk', 'No Risk'])
#plt.legend(bbox_to_anchor=(1.5, 1), loc='upper right', labels=['Risk', 'No Risk'])
plt.savefig('/content/drive/MyDrive/ML4EconProject/resultsNetBenefit.png')
plt.show()

In [ ]:
# Assumption 1: Viable applicants bring in 213$ per year on average
# Assumption 2: Non-viable applicants save us $1,000 for each credit card debt default 